In [1]:
%load_ext memory_profiler

## Run benchmark with

From repo root/base folder

```bash
mprof run python extract_point_from_raster_buffer.py -f srg-dev/test-data/pop_density/pop_density/*.tif -g srg-dev/test-data/1000_testing_points.rds
```

## Run all the cells below to record peak memory and time

In [2]:
import numpy as np
import pandas as pd
# import geopandas as gpd
import rioxarray as riox
import dask.dataframe as dd

from pyproj import Transformer
from shapely.geometry import mapping, Point

start = pd.Timestamp('now')
buffer_value = 10_000

## Run the analysis in a scalable way

load the raster and fill NaNs with 0

In [3]:
myraster = (
    riox.open_rasterio('test-data/apg18e_APPMA_NSW.tif')
    .sel(band=1)
)

In [4]:
myraster = myraster.where(myraster != myraster.rio.nodata, drop=True)

In [5]:
myraster

<xarray.DataArray (y: 1022, x: 1253)>
array([[0.   , 0.   , 0.   , ...,   nan,   nan,   nan],
       [0.   , 0.   , 0.   , ...,   nan,   nan,   nan],
       [0.   , 0.   , 0.   , ...,   nan,   nan,   nan],
       ...,
       [0.   , 1.693, 0.   , ...,   nan,   nan,   nan],
       [0.   , 0.   , 0.   , ...,   nan,   nan,   nan],
       [0.   , 0.   , 0.   , ...,   nan,   nan,   nan]], dtype=float32)
Coordinates:
    band         int64 1
  * x            (x) float64 8.285e+05 8.295e+05 8.305e+05 ... 2.08e+06 2.08e+06
  * y            (y) float64 -3.184e+06 -3.186e+06 ... -4.204e+06 -4.206e+06
    spatial_ref  int64 0
Attributes:
    _FillValue:    -3.4028230607370965e+38
    scale_factor:  1.0
    add_offset:    0.0

In [6]:
"{:,}".format(myraster.data.shape[0] * myraster.data.shape[1])

'1,280,566'

Load points for data extraction and create buffers

In [7]:
transformer = Transformer.from_crs("EPSG:3577", myraster.rio.crs, always_xy=True)

In [8]:
points = (
    pd.read_csv('test-data/1000_testing_points.csv')
    .rename(columns={'X': 'x', 'Y': 'y'})
    .assign(
        lat_lon_tuple = lambda columns: columns[['x', 'y']].apply(lambda row: transformer.transform(row['x'], row['y']), axis=1),
        lat = lambda columns: columns['lat_lon_tuple'].apply(lambda el: el[0]),
        lon = lambda columns: columns['lat_lon_tuple'].apply(lambda el: el[1]),
        points = lambda columns: columns['lat_lon_tuple'].apply(Point),
        points_buffer = lambda columns: columns['points'].apply(lambda x: x.buffer(buffer_value))
    )
)
points = dd.from_pandas(points, npartitions=24)# assign partitions equal to 2 x Nr logical cores in my machine

Memory of dataframe in MB

In [9]:
# def extract_mean_from_buffer(raster, geom):
#     data_points = pd.Series(geom.exterior.coords)
#     values_from_raster = data_points.apply(lambda row: raster.sel(x=row[0], y=row[1], method="nearest").item()).values
#     return values_from_raster.mean()
def extract_mean_from_buffer(raster, geom):
    data_points = geom.exterior.coords
    raster_selection = raster.sel(
        x=[el[0] for el in data_points],
        y=[el[1] for el in data_points],
        method="nearest"
    )
    return np.diag(raster_selection.data).mean()

In [10]:
%%memit
points['extracted_mean'] = points['points_buffer'].apply(
    lambda x: extract_mean_from_buffer(myraster, x),
    meta=float
) * myraster.attrs['scale_factor'] + myraster.attrs['add_offset']

peak memory: 218.91 MiB, increment: 0.45 MiB


In [11]:
points = points.compute(scheduler="processes")

In [12]:
print(f"running time: {pd.Timestamp('now') - start}")

running time: 0 days 00:00:02.238083


In [13]:
points.sample(20)

,x,y,lat_lon_tuple,lat,lon,points,points_buffer,extracted_mean
608,897181.233477,-3.184429e+06,"(897181.23347677, -3184428.5805817773)",897181.233477,-3.184429e+06,POINT (897181.23347677 -3184428.580581777),"POLYGON ((907181.23347677 -3184428.580581777, ...",0.000000
680,905364.791201,-3.184429e+06,"(905364.7912014872, -3184428.5805817773)",905364.791201,-3.184429e+06,POINT (905364.7912014872 -3184428.580581777),POLYGON ((915364.7912014872 -3184428.580581777...,0.000000
879,927983.235468,-3.184429e+06,"(927983.2354684136, -3184428.5805817773)",927983.235468,-3.184429e+06,POINT (927983.2354684136 -3184428.580581777),POLYGON ((937983.2354684136 -3184428.580581777...,0.000000
379,871152.973491,-3.184429e+06,"(871152.9734912114, -3184428.5805817773)",871152.973491,-3.184429e+06,POINT (871152.9734912114 -3184428.580581777),POLYGON ((881152.9734912114 -3184428.580581777...,0.000000
91,838418.742592,-3.184429e+06,"(838418.742592343, -3184428.5805817773)",838418.742592,-3.184429e+06,POINT (838418.742592343 -3184428.580581777),"POLYGON ((848418.742592343 -3184428.580581777,...",0.000000
805,919572.356696,-3.184429e+06,"(919572.3566957876, -3184428.5805817773)",919572.356696,-3.184429e+06,POINT (919572.3566957876 -3184428.580581777),POLYGON ((929572.3566957876 -3184428.580581777...,0.000000
349,867743.157773,-3.184429e+06,"(867743.1577725792, -3184428.5805817773)",867743.157773,-3.184429e+06,POINT (867743.1577725792 -3184428.580581777),POLYGON ((877743.1577725792 -3184428.580581777...,0.000000
592,895362.665093,-3.184429e+06,"(895362.6650934995, -3184428.5805817773)",895362.665093,-3.184429e+06,POINT (895362.6650934995 -3184428.580581777),POLYGON ((905362.6650934995 -3184428.580581777...,0.000000
209,851830.684419,-3.184429e+06,"(851830.6844189627, -3184428.5805817773)",851830.684419,-3.184429e+06,POINT (851830.6844189627 -3184428.580581777),POLYGON ((861830.6844189627 -3184428.580581777...,0.000000
428,876722.339165,-3.184429e+06,"(876722.3391649772, -3184428.5805817773)",876722.339165,-3.184429e+06,POINT (876722.3391649772 -3184428.580581777),POLYGON ((886722.3391649772 -3184428.580581777...,0.000000
